# Planet Features API Python Client Introduction


This tutorial is an introduction to [Planet's](https://www.planet.com) Features API using the official [Python client](https://github.com/planetlabs/planet-client-python), the `Planet` module. The Features API is used to create *Collections* made up of features that follow [OGC-compliant features standards](https://ogcapi.ogc.org/features/). The features are stored as GeoJson format FeatureCollections, and will be then be available to use across the Planet platform, including in Planet Explorer and Features Manager. The features within your feature collection are referred to as *items* within the FeatureCollection.

## Requirements

An account on the [Planet Platform](https://www.planet.com/account/) is required to access any of Planet's API's. If you are not logged in, you will be prompted to do so below in the *SDK Authentication* section.

## Useful links 
* [Planet SDK for Python](https://planet-sdk-for-python.readthedocs.io/en/stable/get-started/quick-start-guide/)
* [Planet Python Client Repo](https://github.com/planetlabs/planet-client-python)
* [Planet Features API Documentation](https://docs.planet.com/develop/apis/features/)

The basic workflow for interaction with the Features API is:
1. Define Feature(s) through filepath or text
2. Create a Collection with a name
3. Add features to the collection
4. Access items in other Planet API calls

Collections and the Features they hold will be viewable in the Planet Platform [Features Manager](https://www.planet.com/features/). You should be able to see all the Features your org has access to.
<br><br>**Note** that when you upload a feature, it is not required to have a title. Your feature may display without a 'Feature Name' in the GUI without a title input when you upload it. This is intended, and does not affect any of the usage of the features we will be creating.

Using this workflow this tutorial will walk you through using a predefined AOI in Cairo, Egypt around Cairo Lake as the feature for our Collection. This Collection will then be used in an order of PlanetScope Sandbox data during May 2022. Planet Sandbox data is included with both trial and paid Planet Accounts.

____

## Set up

In order to interact with the Planet API using the Python client, we need to import the necessary packages and authenticate our Planet account credentials.

### Imports

In [ ]:
import json
import os

import geopandas as gpd
from datetime import datetime
import planet
from planet import Auth, Planet, Session

### SDK Authentication

Your Planet login is used to authenticate and activate the Python SDK. You will be prompted via a link below to login and confirm on the page that the code displayed matches the authorization code printed. If this is your first time accessing the Planet SDK, you will also be prompted first to authorize the SDK access to your Planet account. 
If you would like to know more, please visit the [authentication documentation](https://docs.planet.com/develop/authentication).

In [164]:
# OAuth2 python client authentication
# If you are not already logged in, this will prompt you to open a web browser to log in.

auth = Auth.from_profile('planet-user', save_state_to_storage=False)
auth.ensure_initialized(allow_open_browser=True, allow_tty_prompt=True)

session = Session(auth)
pl = Planet(session)

Opening browser to login.
Confirm the authorization code when prompted: GHHD-VLWB



_____

### Define Features

First, define the FeatureCollection you wish to post to the FeaturesAPI. 

Your FeatureCollection must be:

- Standard GeoJSON FeatureCollection format
- Must use WGS84/EPSG:4326 projection
- Be 2 dimensional
- Contain 1500 vertices or *less*

Full feature rules may be found [here](https://docs.planet.com/develop/apis/features/uploading-and-validating-features/#rules-for-creating-a-feature).
<br> <br>
Below we have defined a FeatureCollection of two areas in Cairo, Egypt. You may use this FeatureCollection to follow along with the tutorial, or import your own from a filepath.

You have a few options to define features to post to your Collections:
- The most simple option is to use a predefined FeatureCollection, so that the Features API can separate each feature into individual items for you.
- You may alternatively pass in just the geometries from feature collections. 
    - There is a free tool [geojson.io](https://geojson.io/) that will allow you to create geoetries and FeatureCollections by hand. You can also draw and download custom AOI geojson's from [Planet Explorer](https://www.planet.com/explorer/).
- You can read a geojson file from a local path, a snippet is provided below to do this. 

Features can have properties and keys beyond just the `geometry` and the `id`. If a feature is uploaded without specifying the `property_id`, all properties of the feature are maintained in the item.


In [ ]:
lake_poly = [[[31.23905453, 29.99579775], [31.26630889, 29.99579775], [31.26630889, 30.01908084], [31.23905453, 30.01908084], [31.23905453, 29.99579775]]]
institute_poly = [[[31.23344423, 30.05109414], [31.25491533, 30.05109414], [31.25491533, 30.06461941], [31.23344423, 30.06461941], [31.23344423, 30.05109414]]]

cairo_fc = {
    "type": "FeatureCollection",
    "features": [
        {
            "id": "0",
            "type": "Feature",
            "properties": {
                "name_str": "area_1",
                "extra_property_1": 'extra property 1 here',
                "extra_property_2": 'extra property 2 here'},
            "geometry": {"coordinates": lake_poly, "type": "Polygon"},
        },
        {
            "id": "1",
            "type": "Feature",
            "properties": {"name_str": "area_2"},
            "geometry": {"coordinates": institute_poly, "type": "Polygon"},
        },
]}

In [ ]:
# Read a geojson from a path
geojson_file = 'PATH TO YOUR GEOJSON FILE'

def load_geojson(filename):
    with open(filename, 'r') as f:
        return json.load(f)

complex_geom = load_geojson(geojson_file)

___

### Name and create your FeatureCollection in the Features API

Before you upload your FeatureCollection items, you need to create the Collection with the Features API. Set your `collection_title`, which will serve as the title or name of the whole FeatureCollection once its uploaded. You may optionally set a description for your Collection. Then we can create our new feature_collection using `pl.features.create_collection`, which returns the newly created Collections's `collection_id`. The `collection_id` will be used to add items to the Collection.

In [ ]:
collection_title = 'cairo_areas'
description = 'FeatureCollection of two areas in Cario, Egypt'

In [187]:
new_collection_id = pl.features.create_collection(title=collection_title, description=description)

In [120]:
new_collection_id

'cairo_areas-proeNRv'

### Post Features to your Collection

After the Collection is created with the Features API, you may post your features from the FeatureCollection we defined above. We will also set the `property_id` to use for the features. 
<br> <br>
The `property_id` is optional, it refers to which key from the `properties` block of the supplied feature(s) to use as the name for the collection. This gives you control of the identifier for items once they are in your collection. Without the `property_id` specified, a random string is generated for you. The example FeatureColleciton uses the property `name_str`, so our features will be identified with the strings `area_1` and `area_2`.

In [183]:
property_id = 'name_str'

In [188]:
new_items = pl.features.add_items(collection_id=new_collection_id, feature = cairo_fc, property_id=property_id)

In [124]:
new_items

['pl:features/my/cairo_areas-proeNRv/area_1-0KZLn6Z',
 'pl:features/my/cairo_areas-proeNRv/area_2-mWVX7Qr']

`add_items` returned a list of strings, which are the `pl reference ids` of the items we just added to the colleciton. These string are what we can use to as the `geometry` input for Orders and Subscriptions later. Before we order imagery with these, we will first explore the other functionalities of the Features python client.

____

### Access your feature Collecitons from the Features API

Basic information (title, description, extent, etc.) about all the feature collections you have access to is available with `list_collections()`

In [ ]:
collections = pl.features.list_collections()
collections_list = [collection for collection in collections]
collections_list

Individual Collection items can be listed with `list_items()`. You can then use the item ids' returned from the items to access the individual items.

We will use the collection_id of the collection we created above, `new_collection_id`:

In [ ]:
items = pl.features.list_items(new_collection_id)
item_list = [item for item in items]
item_list

To get an individual item from the feature collection, you need the `collection_id` as well as the `feature_id` itself. We will be reusing the `collection_id` from before, and using the first item in our list in that collection.

In [138]:
collection_id = new_collection_id
feature_id = item_list[0]['id']

In [ ]:
pl.features.get_item(collection_id, feature_id)

___

### Deleting Collections

Using the `collection_id` you may delete a collection using `delete_collection()`. Be careful when you do this, as there is no confirmation output from this method. 

In [ ]:
coll_id = ''
pl.features.delete_collection(coll_id)

_____

# Using Feature References as input geometries

The `'pl:ref'` property of features is can be used in other Planet APIs to link to the geometry of the feature instead of having the user tracking bounding box and geometry variables. Planet `Subscriptions`, `Orders`, and `Data` APIs are all able to make use of these pl feature reference links. 

Pl ref ids are submitted in the `geometery` argument as a dictionary: `{'type': 'ref', 'content': 'pl:features/my/<collection_id>/<item_id>'}`

Below provided is consilidated code to use the SDK to get a `pl:ref` id from the features API, and then input that id into an order_request. If you are unfamiliar with the mechanics of the `order_request` functions of the SDK, there is an `<order notebook path>` that will walk you through how to use the Planet Python SDK to create and submit Orders.

as well as adding it to a request body for a `request` call. These snippets can be copied into other workflows or scripts you may have to utilize the Features API.

In [150]:
### SDK Order workflow
## example: get the first Collection you have, and use the id of its first item as a geometry in an Order

# list all your collections
collections = pl.features.list_collections()
collections_list = [collection for collection in collections]

# Get the id of the first collection in your list
collection_id = collections_list[0]['id']

collection_items = list(pl.features.list_items(collection_id))
feature_ref = collection_items[1]['properties']['pl:ref']
print(feature_ref)

pl:features/my/cairo_areas-proeNRv/area_1-0KZLn6Z


In [ ]:


prod = planet.order_request.product()
tools = [planet.order_request.clip_tool(feature_ref)]

In [158]:
name = 'test_cairo_order'


order_made = planet.order_request.build_request(name, prod, tools=tools)

In [ ]:
order_made

In [ ]:
pl.orders.create_order?